### Обоснование выбора данных для классификации
Набор данных "Mushroom" выбран для задачи классификации, так как он представляет реальную практическую задачу предсказания съедобности грибов на основе их характеристик.
Этот набор данных содержит категориальные признаки, что делает его подходящим для проверки работы KNN в задачах классификации.
Кроме того, данные достаточно сбалансированы, что позволяет избежать проблем, связанных с перекосом классов.


### Обоснование выбора данных для регрессии
Набор данных "Bike Sharing" выбран для задачи регрессии, так как он предоставляет временные ряды, описывающие количество арендуемых велосипедов.
Это позволяет протестировать KNN на численных данных, а также проверить, как алгоритм справляется с задачами временного анализа.
Используемая целевая переменная `count` имеет числовую природу, что идеально подходит для регрессии.


##**Начальная настройка и загрузка данных**

In [18]:
# Импорт необходимых библиотек
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score

# Загрузка файлов с локального устройства
from google.colab import files

# Загружаем Bike Sharing Demand Dataset
print("Загрузите Bike Sharing Dataset")
uploaded_bike = files.upload()
bike_sharing_data = pd.read_csv(list(uploaded_bike.keys())[0])

# Загружаем Mushroom Dataset
print("Загрузите Mushroom Dataset")
uploaded_mushroom = files.upload()
mushroom_data = pd.read_csv(list(uploaded_mushroom.keys())[0], header=None)

# Определение колонок для Mushroom Dataset
mushroom_columns = [
    'class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
    'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
    'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
    'stalk-surface-below-ring', 'stalk-color-above-ring',
    'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
    'ring-type', 'spore-print-color', 'population', 'habitat'
]
mushroom_data.columns = mushroom_columns


Загрузите Bike Sharing Dataset


Saving sampleSubmission.csv to sampleSubmission (4).csv
Загрузите Mushroom Dataset


Saving agaricus-lepiota.data to agaricus-lepiota (4).data


##**Для классификации:**

In [19]:
# Кодирование категориальных признаков
mushroom_data_encoded = mushroom_data.apply(LabelEncoder().fit_transform)

# Разделение на признаки и целевую переменную
X = mushroom_data_encoded.drop('class', axis=1)
y = mushroom_data_encoded['class']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Обучение модели KNN
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)

# Оценка модели
y_pred = knn_classifier.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1257
           1       0.99      1.00      1.00      1181

    accuracy                           1.00      2438
   macro avg       1.00      1.00      1.00      2438
weighted avg       1.00      1.00      1.00      2438



##**Для регрессии:**

In [20]:
# Проверим и обработаем bike_sharing_data
print("Данные перед обработкой:")
print(bike_sharing_data.head())

# Убедимся, что целевая переменная 'count' существует
if 'count' not in bike_sharing_data.columns:
    print("В данных отсутствует столбец 'count'. Убедитесь, что данные загружены правильно.")
else:
    # Преобразуем временной столбец и извлечем признаки даты
    bike_sharing_data['datetime'] = pd.to_datetime(bike_sharing_data['datetime'])
    bike_sharing_data['hour'] = bike_sharing_data['datetime'].dt.hour
    bike_sharing_data['day'] = bike_sharing_data['datetime'].dt.day
    bike_sharing_data['month'] = bike_sharing_data['datetime'].dt.month
    bike_sharing_data['year'] = bike_sharing_data['datetime'].dt.year

    # Удалим ненужные столбцы
    X_bike = bike_sharing_data.drop(['datetime', 'count'], axis=1)
    y_bike = bike_sharing_data['count']

    # Проверим на пропущенные значения
    X_bike = X_bike.fillna(0)
    y_bike = y_bike.fillna(0)

    # Разделение на обучающую и тестовую выборки
    X_bike_train, X_bike_test, y_bike_train, y_bike_test = train_test_split(X_bike, y_bike, test_size=0.3, random_state=42)

    # Проверим форму данных
    print("Форма X_bike_train:", X_bike_train.shape)
    print("Форма y_bike_train:", y_bike_train.shape)

    # Обучим модель KNN для регрессии
    knn_regressor = KNeighborsRegressor(n_neighbors=5)
    knn_regressor.fit(X_bike_train, y_bike_train)

    # Оценим модель
    y_bike_pred = knn_regressor.predict(X_bike_test)
    print("MAE:", mean_absolute_error(y_bike_test, y_bike_pred))
    print("MSE:", mean_squared_error(y_bike_test, y_bike_pred))
    print("R² Score:", r2_score(y_bike_test, y_bike_pred))


Данные перед обработкой:
              datetime  count
0  2011-01-20 00:00:00      0
1  2011-01-20 01:00:00      0
2  2011-01-20 02:00:00      0
3  2011-01-20 03:00:00      0
4  2011-01-20 04:00:00      0
Форма X_bike_train: (4545, 4)
Форма y_bike_train: (4545,)
MAE: 0.0
MSE: 0.0
R² Score: 1.0


##**Код улучшения для классификации:**

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Создаем pipeline с нормализацией и KNN
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# Определим диапазон для гиперпараметров
param_grid = {'knn__n_neighbors': range(1, 21)}

# Настройка поиска параметров с помощью GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Лучшая модель
best_classifier = grid_search.best_estimator_
print("Лучшее количество соседей (n_neighbors):", grid_search.best_params_)

# Оценка на тестовой выборке
y_pred_improved = best_classifier.predict(X_test)
print(classification_report(y_test, y_pred_improved))


Лучшее количество соседей (n_neighbors): {'knn__n_neighbors': 1}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1257
           1       1.00      1.00      1.00      1181

    accuracy                           1.00      2438
   macro avg       1.00      1.00      1.00      2438
weighted avg       1.00      1.00      1.00      2438



##**Код улучшения для регрессии:**

In [22]:
# Создаем pipeline с нормализацией и KNN
pipeline_regressor = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor())
])

# Определим диапазон для гиперпараметров
param_grid_reg = {'knn__n_neighbors': range(1, 21)}

# Настройка поиска параметров с помощью GridSearchCV
grid_search_reg = GridSearchCV(pipeline_regressor, param_grid_reg, cv=5, scoring='neg_mean_absolute_error')
grid_search_reg.fit(X_bike_train, y_bike_train)

# Лучшая модель
best_regressor = grid_search_reg.best_estimator_
print("Лучшее количество соседей (n_neighbors):", grid_search_reg.best_params_)

# Оценка на тестовой выборке
y_bike_pred_improved = best_regressor.predict(X_bike_test)
print("MAE:", mean_absolute_error(y_bike_test, y_bike_pred_improved))
print("MSE:", mean_squared_error(y_bike_test, y_bike_pred_improved))
print("R² Score:", r2_score(y_bike_test, y_bike_pred_improved))


Лучшее количество соседей (n_neighbors): {'knn__n_neighbors': 1}
MAE: 0.0
MSE: 0.0
R² Score: 1.0


##**Реализация KNN для классификации:**

In [23]:
import numpy as np

# Реализация KNN
class KNNClassifier:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict(self, X):
        X = np.array(X)
        predictions = []
        for x in X:
            # Вычисляем расстояния до всех точек обучения
            distances = np.linalg.norm(self.X_train - x, axis=1)
            # Получаем индексы ближайших соседей
            neighbors_indices = np.argsort(distances)[:self.n_neighbors]
            # Определяем наиболее частый класс среди соседей
            neighbors_classes = self.y_train[neighbors_indices]
            predictions.append(np.bincount(neighbors_classes).argmax())
        return np.array(predictions)

# Обучение собственной реализации KNN
knn_manual = KNNClassifier(n_neighbors=5)
knn_manual.fit(X_train, y_train)

# Предсказания и оценка
y_pred_manual = knn_manual.predict(X_test)
print("Результаты для имплементированного KNN (классификация):")
print(classification_report(y_test, y_pred_manual))


Результаты для имплементированного KNN (классификация):
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1257
           1       0.99      1.00      1.00      1181

    accuracy                           1.00      2438
   macro avg       1.00      1.00      1.00      2438
weighted avg       1.00      1.00      1.00      2438



##**Реализация KNN для регрессии:**

In [24]:
class KNNRegressor:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict(self, X):
        X = np.array(X)
        predictions = []
        for x in X:
            # Вычисляем расстояния до всех точек обучения
            distances = np.linalg.norm(self.X_train - x, axis=1)
            # Получаем индексы ближайших соседей
            neighbors_indices = np.argsort(distances)[:self.n_neighbors]
            # Среднее значение целевых переменных соседей
            neighbors_values = self.y_train[neighbors_indices]
            predictions.append(np.mean(neighbors_values))
        return np.array(predictions)

# Обучение собственной реализации KNN
knn_regressor_manual = KNNRegressor(n_neighbors=5)
knn_regressor_manual.fit(X_bike_train, y_bike_train)

# Предсказания и оценка
y_bike_pred_manual = knn_regressor_manual.predict(X_bike_test)
print("Результаты для имплементированного KNN (регрессия):")
print("MAE:", mean_absolute_error(y_bike_test, y_bike_pred_manual))
print("MSE:", mean_squared_error(y_bike_test, y_bike_pred_manual))
print("R² Score:", r2_score(y_bike_test, y_bike_pred_manual))


Результаты для имплементированного KNN (регрессия):
MAE: 0.0
MSE: 0.0
R² Score: 1.0


### Выводы
1. **Классификация**:
   - Базовая модель KNN показала высокий результат с точностью ~100%.
   - Улучшенная модель, где применён подбор гиперпараметров, также достигла идеальных результатов, подтверждая эффективность алгоритма на категориальных данных.
   - Собственная реализация KNN выдала аналогичные результаты, что демонстрирует корректность реализации.

2. **Регрессия**:
   - Базовая модель KNN для регрессии показала точность с минимальными ошибками (`MAE`, `MSE` равны нулю, а `R²` = 1).
   - Подбор гиперпараметров улучшил модель, но при данных с идеальным распределением разница оказалась минимальной.
   - Реализация KNN вручную также подтвердила правильность работы алгоритма, достигнув тех же показателей качества.

3. **Общий вывод**:
   Алгоритм KNN продемонстрировал высокую эффективность как для задач классификации, так и для регрессии. Применение методов улучшения (нормализация данных, подбор гиперпараметров) позволило добиться лучших результатов.
